In [79]:
import pandas as pd

df = pd.read_csv('../../data/clean/digital_footprints.csv')
df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [80]:
df.shape

(744641, 5)

In [ ]:
df.shape

In [81]:
df['date_time'] = pd.to_datetime(df['date_time'])

In [85]:
df.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [87]:
df = df.sort_values(by=['client_id', 'date_time'])

# Identify the last "start" step for each client_id
last_start_df = df[df['process_step'] == 'start'].groupby('client_id')['date_time'].max().reset_index()

# Merge the last start and last confirm times with the original DataFrame
df = pd.merge(df, last_start_df, on='client_id', how='left', suffixes=('', '_last_start'))

# Correct the filtering condition using parentheses for both conditions
client_last_start = df[(df['date_time'] >= df['date_time_last_start'])]

# Drop the helper columns
client_last_start = client_last_start.drop(columns=['date_time_last_start'])
client_last_start

,client_id,visitor_id,visit_id,process_step,date_time
0,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36
1,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45
2,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31
3,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05
4,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09
...,...,...,...,...,...
744636,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:40:08
744637,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:41:28
744638,9999875,738878760_1556639849,931268933_219402947_599432,step_2,2017-06-01 22:41:47
744639,9999875,738878760_1556639849,931268933_219402947_599432,step_3,2017-06-01 22:44:58


In [89]:
client_last_start.to_csv('../../data/clean/client_last_start.csv')

In [92]:
df_client_info = pd.read_csv('df_client_info.csv')
df_client_info.head()

,Unnamed: 0,client_id,tenure_year,tenure_month,age,gender,number_of_accounts,balance,calls_6_mnth,logons_6_mnth,Variation
0,0,836976,6.0,73.0,60.5,M,2.0,45105.30,6.0,9.0,Test
1,1,2304905,7.0,94.0,58.0,M,2.0,110860.30,6.0,9.0,Control
2,2,1439522,5.0,64.0,32.0,M,2.0,52467.79,6.0,9.0,Test
3,3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test
4,4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control


In [94]:
df_client_info.shape

(70609, 11)

In [96]:
df_client_info = df_client_info.drop(columns=['Unnamed: 0'], errors='ignore')
df_client_info.head()

,client_id,tenure_year,tenure_month,age,gender,number_of_accounts,balance,calls_6_mnth,logons_6_mnth,Variation
0,836976,6.0,73.0,60.5,M,2.0,45105.30,6.0,9.0,Test
1,2304905,7.0,94.0,58.0,M,2.0,110860.30,6.0,9.0,Control
2,1439522,5.0,64.0,32.0,M,2.0,52467.79,6.0,9.0,Test
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control


In [98]:
df_total_info = pd.merge(client_last_start, df_client_info, on='client_id', how='inner')
df_total_info.head()

,client_id,visitor_id,visit_id,process_step,date_time,tenure_year,tenure_month,age,gender,number_of_accounts,balance,calls_6_mnth,logons_6_mnth,Variation
0,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,no participation
1,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,no participation
2,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,no participation
3,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,no participation
4,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0,no participation


In [225]:
df_total_info.to_csv('../../data/clean/df_total_info.csv')

In [102]:
df_total_info.shape

(293124, 14)

In [119]:
df_total_info.isnull().sum()

client_id              0
visitor_id             0
visit_id               0
process_step           0
date_time              0
tenure_year           65
tenure_month          65
age                   70
gender                 0
number_of_accounts    65
balance               65
calls_6_mnth          65
logons_6_mnth         65
Variation              0
dtype: int64

In [188]:
df_total_info.dropna(inplace=True)

In [190]:
df_total_info.isnull().sum()

client_id             0
visitor_id            0
visit_id              0
process_step          0
date_time             0
tenure_year           0
tenure_month          0
age                   0
gender                0
number_of_accounts    0
balance               0
calls_6_mnth          0
logons_6_mnth         0
Variation             0
dtype: int64

In [194]:
df_total_info.shape

(293054, 14)

In [196]:
df_total_info['client_id'].nunique()

69999

In [198]:
df_total_info['Variation'].unique()

array(['no participation', 'Test', 'Control'], dtype=object)

In [200]:
df_test_info = df_total_info[df_total_info['Variation'] == 'Test']
df_control_info = df_total_info[df_total_info['Variation'] == 'Control']
df_no_participation_info = df_total_info[df_total_info['Variation'] == 'no participation']

In [202]:
print(df_test_info.shape)
print(df_control_info.shape)
print(df_no_participation_info.shape)

(115369, 14)
(94197, 14)
(83488, 14)


In [204]:
print(df_test_info['client_id'].nunique())
print(df_control_info['client_id'].nunique())
print(df_no_participation_info['client_id'].nunique())

26672
23391
19936


In [217]:
df_test_info.to_csv('../../data/clean/df_test_info.csv')

In [218]:
df_control_info.to_csv('../../data/clean/df_control_info.csv')

In [219]:
df_no_participation_info.to_csv('../../data/clean/df_no_participation_info.csv')

In [206]:
start_last =  df_test_info[df_test_info['process_step'] == 'start'].groupby('client_id').last()
step_1_last = df_test_info[df_test_info['process_step'] == 'step_1'].groupby('client_id').last()
step_2_last = df_test_info[df_test_info['process_step'] == 'step_2'].groupby('client_id').last()
step_3_last = df_test_info[df_test_info['process_step'] == 'step_3'].groupby('client_id').last()
confirm_last =  df_test_info[df_test_info['process_step'] == 'confirm'].groupby('client_id').last()

clients_started = set(start_last.index)
clients_step_1 = set(step_1_last.index)
clients_step_2 = set(step_2_last.index)
clients_step_3 = set(step_3_last.index)
clients_confirmed = set(confirm_last.index)

# Nombre total de clients ayant commencé
total_clients_started = len(clients_started)
total_clients_step_1 = len(clients_step_1)
total_clients_step_2 = len(clients_step_2)
total_clients_step_3 = len(clients_step_3)
total_clients_confirmed = len(clients_confirmed)

# Calcul du taux de complétion step 1
completion_rate_step_1 = total_clients_step_1 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 1 group test: {completion_rate_step_1:.2%}")

# Calcul du taux de complétion step 2
completion_rate_step_2 = total_clients_step_2 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 2 group test: {completion_rate_step_2:.2%}")

# Calcul du taux de complétion step 3
completion_rate_step_3 = total_clients_step_3 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 3 group test: {completion_rate_step_3:.2%}")

# Calcul du taux de complétion total
completion_rate_total = total_clients_confirmed / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate total group test: {completion_rate_total:.2%}")

Completion rate step 1 group test: 78.24%
Completion rate step 2 group test: 72.51%
Completion rate step 3 group test: 70.62%
Completion rate total group test: 67.12%


In [208]:
start_last = df_control_info[df_control_info['process_step'] == 'start'].groupby('client_id').last()
step_1_last = df_control_info[df_control_info['process_step'] == 'step_1'].groupby('client_id').last()
step_2_last = df_control_info[df_control_info['process_step'] == 'step_2'].groupby('client_id').last()
step_3_last = df_control_info[df_control_info['process_step'] == 'step_3'].groupby('client_id').last()
confirm_last = df_control_info[df_control_info['process_step'] == 'confirm'].groupby('client_id').last()

clients_started = set(start_last.index)
clients_step_1 = set(step_1_last.index)
clients_step_2 = set(step_2_last.index)
clients_step_3 = set(step_3_last.index)
clients_confirmed = set(confirm_last.index)

# Nombre total de clients ayant commencé
total_clients_started = len(clients_started)
total_clients_step_1 = len(clients_step_1)
total_clients_step_2 = len(clients_step_2)
total_clients_step_3 = len(clients_step_3)
total_clients_confirmed = len(clients_confirmed)

# Calcul du taux de complétion step 1
completion_rate_step_1 = total_clients_step_1 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 1 group control: {completion_rate_step_1:.2%}")

# Calcul du taux de complétion step 2
completion_rate_step_2 = total_clients_step_2 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 2 group control: {completion_rate_step_2:.2%}")

# Calcul du taux de complétion step 3
completion_rate_step_3 = total_clients_step_3 / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate step 3 group control: {completion_rate_step_3:.2%}")

# Calcul du taux de complétion total
completion_rate_total = total_clients_confirmed / total_clients_started if total_clients_started > 0 else 0
print(f"Completion rate total group control: {completion_rate_total:.2%}")

Completion rate step 1 group control: 75.44%
Completion rate step 2 group control: 66.32%
Completion rate step 3 group control: 63.41%
Completion rate total group control: 58.50%
